## Confidentiality

This notebook is downloaded from Tensorflow and is for demonstrational purposes only.

Please do not copy or distribute this notebook.

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file.

Downloading and installing TensorFlow 2. Importing TensorFlow into your program:


In [ ]:
import tensorflow as tf

Loading and preparing the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Converting the samples from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Building the `tf.keras.Sequential` model by stacking layers. Choosing an optimizer and loss function for training:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.3081565 ,  0.07416787,  0.06665076,  0.19606952, -1.1165055 ,
         0.11613074,  0.42023766,  0.9980431 , -0.05100212, -0.44132158]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.11338348, 0.08972838, 0.08905641, 0.10136104, 0.0272789 ,
        0.09357376, 0.12683117, 0.22602879, 0.07917156, 0.05358653]],
      dtype=float32)

Note: It's possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

2.3690052

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2987 - accuracy: 0.9126
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1427 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1064 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0892 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0749 - accuracy: 0.9765


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0723 - accuracy: 0.9790


[0.07234793156385422, 0.9789999723434448]

The image classifier is now trained to ~98% accuracy on this dataset. 

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.07516321e-07, 2.48641219e-09, 3.35210643e-05, 3.04697867e-04,
        4.54583940e-11, 4.83312448e-08, 8.60110729e-14, 9.99659181e-01,
        3.15917333e-07, 2.13084127e-06],
       [6.12456574e-09, 4.35758775e-05, 9.99942064e-01, 1.36110875e-05,
        6.23493753e-16, 4.85674093e-07, 1.76569674e-08, 2.43354815e-11,
        2.33227297e-07, 1.31496582e-15],
       [2.66468135e-07, 9.98536944e-01, 1.26353829e-04, 2.02665124e-05,
        3.10943069e-05, 4.55603185e-05, 3.66795757e-05, 3.70575348e-04,
        8.32121586e-04, 2.00805673e-07],
       [9.99875665e-01, 5.69708336e-10, 9.18710721e-05, 3.05947140e-07,
        1.72580439e-09, 1.45515505e-05, 2.44360422e-06, 6.83008147e-06,
        5.61715430e-09, 8.28743759e-06],
       [1.77698283e-07, 8.96994534e-10, 2.90815905e-07, 6.94513735e-09,
        9.99476373e-01, 2.35387972e-07, 1.04129481e-06, 3.28384376e-05,
        3.12300722e-06, 4.86033241e-04]], dtype=float32)>